# Get League Table and Stats

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import numpy as np
import time
import random
#imports

In [50]:
# URL of the website
url = "https://fbref.com/en/comps/10/Championship-Stats"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
    # Parse the HTML content of the page
soup = BeautifulSoup(response.text, "html.parser")

    # Find the table using its HTML class
table = soup.find("table", {"class": "stats_table"})

    # Extract table data into a list of lists
table_data = []
for row in table.find_all("tr"):
    row_data = [cell.text.strip() for cell in row.find_all(["th", "td"])]
    table_data.append(row_data)

    # Create a DataFrame from the list of lists
columns = table_data[0]  # Assuming the first row contains column headers
data = table_data[1:]
league_table_df = pd.DataFrame(data, columns=columns)

In [51]:
response

<Response [200]>

# Get Fixtures 

In [52]:
# URL of the website
url = "https://fbref.com/en/comps/10/schedule/Championship-Scores-and-Fixtures"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table containing fixtures
    table = soup.find("table", {"class": "stats_table"})

    # Extract data into a list of dictionaries
    fixtures_data = []
    for row in table.find_all("tr")[1:]:  # Skip the header row
        columns = row.find_all(["th", "td"])
        
         # Extracting specific columns based on the structure of the table
        fixture_type = columns[0].text.strip()
        gameweek = columns[1].text.strip()
        day_of_week = columns[2].text.strip()
        date = columns[3].text.strip()
        match_time = columns[4].text.strip()
        home_team = columns[5].text.strip()
        home_xG = columns[6].text.strip()
        result = columns[7].text.strip()
        away_xG = columns[8].text.strip()
        away_team = columns[9].text.strip()
        attendance = columns[10].text.strip()
        stadium = columns[11].text.strip()
        referee = columns[12].text.strip()

        fixture_info = {
            "fixture_type": fixture_type,
            "gameweek": gameweek,
            "day_of_week": day_of_week,
            "date": date,
            "match_time": match_time,
            "home_team": home_team,
            "home_xG": home_xG,
            "result": result,
            "away_xG": away_xG,
            "away_team": away_team,
            "attendance": attendance,
            "stadium": stadium,
            "referee": referee
        }

        fixtures_data.append(fixture_info)

    # Create a DataFrame from the list of dictionaries
    fixtures_table = pd.DataFrame(fixtures_data)

In [53]:
fixtures_table['is_game_complete'] = np.where(len(fixtures_table['result']) > 0, 1, 0)
#Checking to see if game is complete... if the result column is populated at all, it's been played

# Getting Game IDs

In [56]:
# URL of the website
url = "https://fbref.com/en/comps/10/schedule/Championship-Scores-and-Fixtures"

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <td> elements with data-stat="score"
score_cells = soup.find_all('td', {'data-stat': 'score'})

# Extract game IDs from the <a> tags within the score_cells, the third element is the game ID
game_ids = [cell.find('a')['href'].split('/')[3].strip() for cell in score_cells if cell.find('a')]

# Use a set to ensure uniqueness and convert back to a list
game_ids = list(set(game_ids))

In [57]:
# game_ids = game_ids[0:4]
#just use five to test

In [58]:
from collections import Counter
Counter(game_ids)

Counter({'ae75c51d': 1,
         'c9353098': 1,
         'd11e033a': 1,
         '86121494': 1,
         '773f8407': 1,
         '43feef12': 1,
         '5f3ff379': 1,
         'bc26816d': 1,
         '4f28a398': 1,
         'a6aa254b': 1,
         'e669279a': 1,
         '29b786f2': 1,
         'd098d4ce': 1,
         '15254aac': 1,
         'ab2ff3b6': 1,
         '3aa56639': 1,
         '293773d2': 1,
         '336642c0': 1,
         '20c1e029': 1,
         '2d45672e': 1,
         'b01a9d49': 1,
         '01f668c2': 1,
         '104d2548': 1,
         '8ecf97e4': 1,
         '7a3e7fc1': 1,
         'b38c790a': 1,
         '97231601': 1,
         '488ad30a': 1,
         '770ecb91': 1,
         '9cf43d69': 1,
         '6b967504': 1,
         'bf2e6470': 1,
         '6bc56228': 1,
         '9ba0b2e5': 1,
         '848e95d4': 1,
         '8279c524': 1,
         'ed42701c': 1,
         '329be1f2': 1,
         '3dd36804': 1,
         '702dd9db': 1,
         '4db419ee': 1,
         '123c47

# Scraping Shot Data

In [59]:
all_data = []

for unique_id in game_ids:
    # URL of the page to scrape
    url = f'https://fbref.com/en/matches/{unique_id}/'

    # Get the game ID from the URL
    game_id = url.split("/")[-2]

    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the Shots table on the page
    shots_table = soup.find("div", {"id": "switcher_shots"})

    # Create empty lists to store data and create the columns
    data = []
    headers = ['game_id', 'minute', 'player', 'team', 'xG', 'psxG', 'result', 'distance', 'body_part', 'notes', 'sca_1_player', 'event_1', 'sca_2_player', 'event_2']

    # Extract table rows
    rows = shots_table.find("tbody").find_all("tr")
    for row in rows:
        row_data = [game_id] + [cell.get_text().strip() for cell in row.find_all(["th", "td"])]
        data.append(row_data)

    # Find the home and away teams and formations
    team_headers = soup.find_all("th", {"colspan": "2"})
    teams = [header.text.strip().split(" (")[0] for header in team_headers if "(" in header.text.strip()]
    formations = [header.text.strip().split(" (")[1][:-1] for header in team_headers if "(" in header.text.strip()]

    # Extracting the home and away teams and formations
    home_team = teams[0]
    away_team = teams[1]
    home_formation = formations[0]
    away_formation = formations[1]

    # Find the location and match date
    location = soup.find("div", {"class": "scorebox_meta"}).find_all("small")[3].text
    match_date = soup.find("span", class_="venuetime").get("data-venue-date", None)

    # Append additional columns to headers
    headers.extend(['home_team', 'away_team', 'home_formation', 'away_formation', 'location', 'match_date'])

    # Append data to all_data with additional columns
    for row in data:
        row.extend([home_team, away_team, home_formation, away_formation, location, match_date])
        all_data.append(row)
        
        
    time.sleep(5)
        
# Create a DataFrame from the scraped data
shots_df = pd.DataFrame(all_data, columns=headers)

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
shots_df

In [30]:
# Add Home and away shots, location, hard-code competition
shots_df['is_home_shot'] = np.where(shots_df['team'] == shots_df['home_team'], 1, 0)
shots_df['is_away_shot'] = np.where(shots_df['team'] ==  shots_df['away_team'], 1, 0)
shots_df['competition'] = 'English Championship'

KeyError: 'home_team'

In [ ]:
# Define a function to calculate total score for each game
def calculate_total_score(group, shot):
    total_score = 0
    total_scores = []

    # Flag to indicate whether the current row is after a 'Goal'
    after_goal = False

    for idx, (result, is_shot) in enumerate(zip(group, shot)):
        # Check if the current row is after a 'Goal'
        if after_goal:
            # Increment the total score
            total_score += 1
            # Reset the flag
            after_goal = False

        # Check if the current result is 'Goal' and the corresponding shot is 1
        if result == 'Goal' and is_shot == 1:
            # Set the flag to True for the next iteration
            after_goal = True
        
        # Append the current total score
        total_scores.append(total_score)

    return total_scores

# Group by game_id and apply the function to calculate total score for home team
shots_df['home_score'] = shots_df.groupby('game_id')['result'].transform(lambda x: calculate_total_score(x, shots_df['is_home_shot']))

# Group by game_id and apply the function to calculate total score for away team
shots_df['away_score'] = shots_df.groupby('game_id')['result'].transform(lambda x: calculate_total_score(x, shots_df['is_away_shot']))


In [ ]:
#Creating a nil nil column using numpy where and logical_and operators
shots_df['is_nil_nil'] = np.where(np.logical_and(shots_df['home_score'] == 0, shots_df['away_score'] == 0), 1, 0)

In [ ]:
#Adding opposing team -- Whenever it's a home shot we want the away team, otherwise (away shot) we want the home team
shots_df['opposing_team'] = np.where(shots_df['is_home_shot'] == 1, shots_df['away_team'], shots_df['home_team'])

In [ ]:
#Replace blank values with nulls
shots_df['xG'] = shots_df['xG'].replace('', np.nan)

#Then drop them
shots_df = shots_df.dropna(subset=['xG'])

In [ ]:
shots_df['xG'] = shots_df['xG'].astype('float64') 

# Adding xG and xGA while nil-nil to league_table_df

In [ ]:
# Filter shots_df where is_nil_nil is 0
filtered_shots_df = shots_df[shots_df['is_nil_nil'] == 1]

# Calculate the sum of 'xG' for each 'team'
team_xG_sum = filtered_shots_df.groupby('team')['xG'].sum().reset_index()

# Create a dictionary to map team names to their respective summed 'xG' values
team_xG_dict = dict(zip(team_xG_sum['team'], team_xG_sum['xG']))

# Add 'nil_nil_xG' column to league_table_df by mapping team names to their summed 'xG' values
league_table_df['nil_nil_xG'] = league_table_df['Squad'].map(team_xG_dict)

# Fill NaN values with 0 if any team didn't have any 'xG' when is_nil_nil is 0
league_table_df['nil_nil_xG'].fillna(0, inplace=True)

# xGA

# Calculate the sum of 'xGA' for each 'team'
team_xGA_sum = filtered_shots_df.groupby('opposing_team')['xG'].sum().reset_index()

# Create a dictionary to map team names to their respective summed 'xGA' values -- AKA xG of opposing_team
team_xGA_dict = dict(zip(team_xGA_sum['opposing_team'], team_xGA_sum['xG']))

# Add 'nil_nil_xGA' column to league_table_df by mapping team names to their summed 'xGA' values
league_table_df['nil_nil_xGA'] = league_table_df['Squad'].map(team_xGA_dict)

# Fill NaN values with 0 if any team didn't have any 'xG' when is_nil_nil is 0
league_table_df['nil_nil_xGA'].fillna(0, inplace=True)

In [ ]:
league_table_df

# To Do

1. Add average xG and xGA per match while nil nil